In [2]:
bins = [155,165,175,185]

def getBin(num):
    out = len(bins)
    for i, b in enumerate(bins):
        if(num<b):
            out = i
            break
    return out        
        
print(getBin(150))     
print(getBin(160))
print(getBin(170))
print(getBin(180))
print(getBin(190))

0
1
2
3
4


In [3]:
def serialHist(array):
    out = [0 for i in range(len(bins)+1)]
    for num in array:
        out[getBin(num)] += 1    
    return out

import numpy as np
values = np.random.normal(170,10,1024)
serialHist(values)

[62, 254, 393, 246, 69]

###Per-Thread Privatized (Local) Histograms, then Reduce

128 items --- 8 threads --- 3 bins

---> each thread gets 16 items

each thread initializes 3 bins in local registers

Then we need to use REDUCTION to combine these 8 local histograms into a single global histogram


In [23]:
def privateHist(array, numThreads):
    n = len(array)
    if(n%numThreads != 0):
        raise IndexError("error, must evenly divide")
    blockSize = n//numThreads    
    numBins = len(bins)+1
    global_out = [[0 for i in range(numBins)]for j in range(numThreads)]
    for i in range(numThreads):
        privateHist = global_out[i]
        for j in range(blockSize):
            num = array[j+i*blockSize]
            privateHist[getBin(num)] += 1
    return global_out
x = privateHist(values, 8)

In [31]:
x = np.array(x)
x.sum(0)

array([ 62, 254, 393, 246,  69])

In [29]:
def myReduce(array):
    n = len(array)
    out = [i for i in array]
    s = 0
    shift = 1<<s
    while(shift<n):
        s += 1
        for i in range(n//(2*s)):
            out[2*s*i] = out[2*s*i]+out[2*s*i+shift]
        shift = 1<<s
    return out[0]
    
[myReduce(x[:,i]) for i in range(len(bins)+1)]  


[62, 254, 393, 246, 69]